# Building up

In [1]:
import numpy as np
import time
time_start = time.time()
X = np.genfromtxt('00_Data/train_X_dog_cat.csv', delimiter=',')
y = np.genfromtxt('00_Data/train_y_dog_cat.csv', delimiter=',')
print(X.shape)

n = X.shape[0]
d = X.shape[1]
print(y.shape)
time_end = time.time()
print("Done loading training data, using time:(min)", (time_end-time_start)/60)

(1953, 3072)
(1953,)
Done loading training data, using time:(min) 0.08970758517583212


In [2]:
X_test = np.genfromtxt('00_Data/test_X_dog_cat.csv', delimiter=',')
y_test = np.genfromtxt('00_Data/test_y_dog_cat.csv', delimiter=',')
m = X_test.shape[0]
print(m)

2000


In [3]:
from sklearn import preprocessing
scaler = preprocessing.MinMaxScaler().fit(X)
X = scaler.transform(X)
X_test = scaler.transform(X_test)


In [20]:
def linear_kernel(x1,x2):
    # x1 = (n,d) X2 = (m,d)
    return(np.dot(x1,x2.T))

def inhomogeneous_poly_kernel(x1,x2):
    n = x1.shape[0]
    m = x2.shape[0]
    return np.power(np.ones(shape=(n,m)) + np.dot(x1,x2.T), 5)

                  
def gaussian_kernel(x1,x2,sigma=10):
    n = x1.shape[0]
    m = x2.shape[0]
    K = np.zeros(shape=(n,m))
    for i in range(n):
        for j in range(m):
            K[i][j] = gaussian(x1[i],x2[j],sigma)
    return (K)

def gaussian(x,y,sigma):
    return np.exp((-(np.linalg.norm(x-y,2)**2))/(2*sigma**2))

In [5]:
def evaluation(ans, y):
    right = 0
    num = ans.shape[0]
    for i in range(ans.shape[0]):
        if (ans[i] - 0.5)*y[i] > 0:
            right += 1
    return(right/float(num))

In [6]:
def tolerable(delta,tol):
    from numpy import linalg as LA
    if LA.norm(delta,1) > tol:
        return False
    else:
        return True

In [7]:
def probability(alpha, Ki):
    s = np.dot(alpha, Ki)
    return(1/(1+np.exp(-s)))

def train(K,y,max_iter,batch_size,regu_para, eta,tol):
    time_start = time.time()
    #alpha = np.random.uniform(size=(n))
    alpha = np.zeros(n)
    for t in range(max_iter):
        samples = np.random.permutation(n)[:batch_size]
        g = regu_para * alpha
        #g = np.zeros(n)
        for i in samples:
            pi = probability(alpha, K[i])
            g[i] = g[i] + (pi - (y[i]+1)/2.0)
        alpha = alpha - eta*g
        if tolerable(g, tol):
            break
        if t % 2000 == 0:
            print("iter %d train_accuracy:%f"%(t,evaluation(probability(alpha,K), y)))
    time_end = time.time()
    print("Done training, using time:(min)", 
          (time_end-time_start)/60)
    return(alpha)

In [8]:
def experiment(kernel, X,y,X_test,y_test,regu_para, eta,tol= 0.1, max_iter=50000, batch_size=640):
    K = kernel(X,X)
    K_test = kernel(X,X_test)
    print("Done kernel generating")
    alpha = train(K,y, max_iter,batch_size,regu_para, eta,tol)
    print("Done training")
    print(alpha)
    print("training result")
    ans = probability(alpha,K)
    #print(ans)
    print(evaluation(ans, y))
    #print(y)
    print("test result")
    ans = probability(alpha,K_test)
    #print(ans)
    print(evaluation(ans, y_test))
    #print(y_test)

# Linear kernel exps

In [28]:
experiment(linear_kernel, X, y, X_test, y_test, regu_para=0, eta=0.01, tol=0.1)

Done kernel generating
iter 0 train_accuracy:0.520225


/home/y247xie/.local/lib/python3.5/site-packages/ipykernel_launcher.py:3: RuntimeWarning: overflow encountered in exp
  This is separate from the ipykernel package so we can avoid doing imports until


iter 2000 train_accuracy:0.555044
iter 4000 train_accuracy:0.589350
iter 6000 train_accuracy:0.593446
iter 8000 train_accuracy:0.624680
iter 10000 train_accuracy:0.665643
iter 12000 train_accuracy:0.629288
iter 14000 train_accuracy:0.621608
iter 16000 train_accuracy:0.697901
iter 18000 train_accuracy:0.627240
iter 20000 train_accuracy:0.720430
iter 22000 train_accuracy:0.710701
iter 24000 train_accuracy:0.661546
iter 26000 train_accuracy:0.632873
iter 28000 train_accuracy:0.604199
iter 30000 train_accuracy:0.584229
iter 32000 train_accuracy:0.654378
iter 34000 train_accuracy:0.636457
iter 36000 train_accuracy:0.675371
iter 38000 train_accuracy:0.584741
iter 40000 train_accuracy:0.719406
iter 42000 train_accuracy:0.623656
iter 44000 train_accuracy:0.725550
iter 46000 train_accuracy:0.610343
iter 48000 train_accuracy:0.751152
Done training, using time:(min) 2.5652570009231566
Done training
[-24.72967924 -82.53314759  -0.45824203 ...  71.2698941   77.80467792
  83.31726411]
training resul

In [ ]:
experiment(linear_kernel, X, y, X_test, y_test, regu_para=1, eta=0.01, tol=0.01,max_iter=20000)

In [29]:
experiment(linear_kernel, X, y, X_test, y_test, regu_para=0, eta=0.001, tol=0.1)

Done kernel generating
iter 0 train_accuracy:0.524322
iter 2000 train_accuracy:0.568356
iter 4000 train_accuracy:0.577573
iter 6000 train_accuracy:0.579109
iter 8000 train_accuracy:0.619048
iter 10000 train_accuracy:0.633385
iter 12000 train_accuracy:0.570405
iter 14000 train_accuracy:0.589350
iter 16000 train_accuracy:0.669739
iter 18000 train_accuracy:0.606759
iter 20000 train_accuracy:0.658986
iter 22000 train_accuracy:0.646697
iter 24000 train_accuracy:0.707117
iter 26000 train_accuracy:0.713262
iter 28000 train_accuracy:0.640553
iter 30000 train_accuracy:0.660522
iter 32000 train_accuracy:0.738863
iter 34000 train_accuracy:0.654378
iter 36000 train_accuracy:0.640041
iter 38000 train_accuracy:0.646185
iter 40000 train_accuracy:0.627752
iter 42000 train_accuracy:0.738351
iter 44000 train_accuracy:0.624680
iter 46000 train_accuracy:0.717870
iter 48000 train_accuracy:0.708141
Done training, using time:(min) 2.2449634432792664
Done training
[-2.55404882 -8.23323814 -0.05373824 ...  7.1

In [30]:
experiment(linear_kernel, X, y, X_test, y_test, regu_para=0, eta=0.0001, tol=0.1)

Done kernel generating
iter 0 train_accuracy:0.520225
iter 2000 train_accuracy:0.573989
iter 4000 train_accuracy:0.589350
iter 6000 train_accuracy:0.585765
iter 8000 train_accuracy:0.628776
iter 10000 train_accuracy:0.636969
iter 12000 train_accuracy:0.589350
iter 14000 train_accuracy:0.619048
iter 16000 train_accuracy:0.652330
iter 18000 train_accuracy:0.619048
iter 20000 train_accuracy:0.658986
iter 22000 train_accuracy:0.704045
iter 24000 train_accuracy:0.644649
iter 26000 train_accuracy:0.737327
iter 28000 train_accuracy:0.651818
iter 30000 train_accuracy:0.704557
iter 32000 train_accuracy:0.758833
iter 34000 train_accuracy:0.693804
iter 36000 train_accuracy:0.630824
iter 38000 train_accuracy:0.663082
iter 40000 train_accuracy:0.694828
iter 42000 train_accuracy:0.706605
iter 44000 train_accuracy:0.714286
iter 46000 train_accuracy:0.692780
iter 48000 train_accuracy:0.691244
Done training, using time:(min) 2.2375803788503013
Done training
[-0.25375142 -0.80916862 -0.01319075 ...  0.7

In [31]:
experiment(linear_kernel, X, y, X_test, y_test, regu_para=10, eta=0.001, tol=0.1)

Done kernel generating
iter 0 train_accuracy:0.520225
iter 2000 train_accuracy:0.520737
iter 4000 train_accuracy:0.520225
iter 6000 train_accuracy:0.519713
iter 8000 train_accuracy:0.519713
iter 10000 train_accuracy:0.521249
iter 12000 train_accuracy:0.522273
iter 14000 train_accuracy:0.520737
iter 16000 train_accuracy:0.519201
iter 18000 train_accuracy:0.519713
iter 20000 train_accuracy:0.520225
iter 22000 train_accuracy:0.519713
iter 24000 train_accuracy:0.520225
iter 26000 train_accuracy:0.520225
iter 28000 train_accuracy:0.521761
iter 30000 train_accuracy:0.521761
iter 32000 train_accuracy:0.519713
iter 34000 train_accuracy:0.520225
iter 36000 train_accuracy:0.522273
iter 38000 train_accuracy:0.521761
iter 40000 train_accuracy:0.519713
iter 42000 train_accuracy:0.520737
iter 44000 train_accuracy:0.520225
iter 46000 train_accuracy:0.521761
iter 48000 train_accuracy:0.520737
Done training, using time:(min) 2.2182202696800233
Done training
[-0.00049907 -0.01286958 -0.01324424 ...  0.0

In [32]:
experiment(linear_kernel, X, y, X_test, y_test, regu_para=30, eta=0.001, tol=0.1)

Done kernel generating
iter 0 train_accuracy:0.520225
iter 2000 train_accuracy:0.520225
iter 4000 train_accuracy:0.520225
iter 6000 train_accuracy:0.520225
iter 8000 train_accuracy:0.520225
iter 10000 train_accuracy:0.520225
iter 12000 train_accuracy:0.520225
iter 14000 train_accuracy:0.520225
iter 16000 train_accuracy:0.520225
iter 18000 train_accuracy:0.520225
iter 20000 train_accuracy:0.520225
iter 22000 train_accuracy:0.520225
iter 24000 train_accuracy:0.520225
iter 26000 train_accuracy:0.520225
iter 28000 train_accuracy:0.520225
iter 30000 train_accuracy:0.520225
iter 32000 train_accuracy:0.520225
iter 34000 train_accuracy:0.520225
iter 36000 train_accuracy:0.520225
iter 38000 train_accuracy:0.520225
iter 40000 train_accuracy:0.520225
iter 42000 train_accuracy:0.520225
iter 44000 train_accuracy:0.520225
iter 46000 train_accuracy:0.520225
iter 48000 train_accuracy:0.520225
Done training, using time:(min) 2.213572013378143
Done training
[-0.00146557 -0.00285996 -0.00867031 ...  0.00

In [33]:
experiment(linear_kernel, X, y, X_test, y_test, regu_para=1, eta=0.001, tol=0.1)

Done kernel generating
iter 0 train_accuracy:0.534562
iter 2000 train_accuracy:0.533026
iter 4000 train_accuracy:0.534562
iter 6000 train_accuracy:0.539171
iter 8000 train_accuracy:0.549411
iter 10000 train_accuracy:0.535074
iter 12000 train_accuracy:0.550435
iter 14000 train_accuracy:0.534562
iter 16000 train_accuracy:0.545827
iter 18000 train_accuracy:0.554019
iter 20000 train_accuracy:0.542755
iter 22000 train_accuracy:0.554019
iter 24000 train_accuracy:0.538658
iter 26000 train_accuracy:0.536610
iter 28000 train_accuracy:0.541219
iter 30000 train_accuracy:0.551971
iter 32000 train_accuracy:0.562724
iter 34000 train_accuracy:0.537122
iter 36000 train_accuracy:0.540195
iter 38000 train_accuracy:0.545827
iter 40000 train_accuracy:0.542243
iter 42000 train_accuracy:0.545315
iter 44000 train_accuracy:0.560164
iter 46000 train_accuracy:0.550947
iter 48000 train_accuracy:0.537634
Done training, using time:(min) 2.1733374913533527
Done training
[-0.15437237 -0.15523217 -0.006991   ...  0.1

In [ ]:
experiment(linear_kernel, X, y, X_test, y_test, regu_para=0.1, eta=0.01, tol=0.1)

In [ ]:
experiment(linear_kernel, X, y, X_test, y_test, regu_para=0.3, eta=0.01, tol=0.1)

In [ ]:
experiment(linear_kernel, X, y, X_test, y_test, regu_para=0.01, eta=0.01, tol=0.1)

# gaussian kernel exps

## sigma = 30

In [9]:
experiment(gaussian_kernel, X, y, X_test, y_test, regu_para=0.1, eta=0.01, tol=0.1)

Done kernel generating
iter 0 train_accuracy:0.520225
iter 2000 train_accuracy:0.655914
iter 4000 train_accuracy:0.661034
iter 6000 train_accuracy:0.670763
iter 8000 train_accuracy:0.670763
iter 10000 train_accuracy:0.671275
iter 12000 train_accuracy:0.663082
iter 14000 train_accuracy:0.668203
iter 16000 train_accuracy:0.669739
iter 18000 train_accuracy:0.664107
iter 20000 train_accuracy:0.661034
iter 22000 train_accuracy:0.668715
iter 24000 train_accuracy:0.666667
iter 26000 train_accuracy:0.668715
iter 28000 train_accuracy:0.671787
iter 30000 train_accuracy:0.668715
iter 32000 train_accuracy:0.662058
iter 34000 train_accuracy:0.662570
iter 36000 train_accuracy:0.668715
iter 38000 train_accuracy:0.669227
iter 40000 train_accuracy:0.661546
iter 42000 train_accuracy:0.670251
iter 44000 train_accuracy:0.673835
iter 46000 train_accuracy:0.671275
iter 48000 train_accuracy:0.662570
Done training, using time:(min) 2.297707998752594
Done training
[-1.11217638 -1.81742875 -1.02836677 ...  1.87

In [10]:
experiment(gaussian_kernel, X, y, X_test, y_test, regu_para=1, eta=0.01, tol=0.1)

Done kernel generating
iter 0 train_accuracy:0.520225
iter 2000 train_accuracy:0.610855
iter 4000 train_accuracy:0.597030
iter 6000 train_accuracy:0.613415
iter 8000 train_accuracy:0.615463
iter 10000 train_accuracy:0.610343
iter 12000 train_accuracy:0.599078
iter 14000 train_accuracy:0.609831
iter 16000 train_accuracy:0.611879
iter 18000 train_accuracy:0.614439
iter 20000 train_accuracy:0.615463
iter 22000 train_accuracy:0.610343
iter 24000 train_accuracy:0.612903
iter 26000 train_accuracy:0.612903
iter 28000 train_accuracy:0.610343
iter 30000 train_accuracy:0.613927
iter 32000 train_accuracy:0.616999
iter 34000 train_accuracy:0.611367
iter 36000 train_accuracy:0.614951
iter 38000 train_accuracy:0.580133
iter 40000 train_accuracy:0.610855
iter 42000 train_accuracy:0.608807
iter 44000 train_accuracy:0.609831
iter 46000 train_accuracy:0.579109
iter 48000 train_accuracy:0.601639
Done training, using time:(min) 2.268279695510864
Done training
[-0.11963902 -0.1592028  -0.14110277 ...  0.15

In [11]:
experiment(gaussian_kernel, X, y, X_test, y_test, regu_para=10, eta=0.01, tol=0.1)

Done kernel generating
iter 0 train_accuracy:0.520225
iter 2000 train_accuracy:0.592422
iter 4000 train_accuracy:0.559140
iter 6000 train_accuracy:0.589350
iter 8000 train_accuracy:0.566308
iter 10000 train_accuracy:0.592422
iter 12000 train_accuracy:0.577061
iter 14000 train_accuracy:0.486431
iter 16000 train_accuracy:0.554531
iter 18000 train_accuracy:0.544291
iter 20000 train_accuracy:0.550947
iter 22000 train_accuracy:0.569380
iter 24000 train_accuracy:0.554531
iter 26000 train_accuracy:0.546339
iter 28000 train_accuracy:0.550435
iter 30000 train_accuracy:0.565796
iter 32000 train_accuracy:0.547875
iter 34000 train_accuracy:0.593958
iter 36000 train_accuracy:0.596006
iter 38000 train_accuracy:0.537122
iter 40000 train_accuracy:0.570405
iter 42000 train_accuracy:0.550435
iter 44000 train_accuracy:0.530978
iter 46000 train_accuracy:0.595494
iter 48000 train_accuracy:0.522273
Done training, using time:(min) 2.2915820360183714
Done training
[-0.01003128 -0.01148967 -0.01567461 ...  0.0

In [12]:
experiment(gaussian_kernel, X, y, X_test, y_test, regu_para=0, eta=0.01, tol=0.1)

Done kernel generating
iter 0 train_accuracy:0.520225
iter 2000 train_accuracy:0.699949
iter 4000 train_accuracy:0.741423
iter 6000 train_accuracy:0.775218
iter 8000 train_accuracy:0.784434
iter 10000 train_accuracy:0.815668
iter 12000 train_accuracy:0.817716
iter 14000 train_accuracy:0.832053
iter 16000 train_accuracy:0.838710
iter 18000 train_accuracy:0.849462
iter 20000 train_accuracy:0.848950
iter 22000 train_accuracy:0.862775
iter 24000 train_accuracy:0.868408
iter 26000 train_accuracy:0.873528
iter 28000 train_accuracy:0.880696
iter 30000 train_accuracy:0.885817
iter 32000 train_accuracy:0.893497
iter 34000 train_accuracy:0.897081
iter 36000 train_accuracy:0.902202
iter 38000 train_accuracy:0.905786
iter 40000 train_accuracy:0.909882
iter 42000 train_accuracy:0.916027
iter 44000 train_accuracy:0.919611
iter 46000 train_accuracy:0.922171
iter 48000 train_accuracy:0.921147
Done training, using time:(min) 2.246520713965098
Done training
[-30.1698246  -66.31461786 -50.7753847  ...  7

In [13]:
experiment(gaussian_kernel, X, y, X_test, y_test, regu_para=0.3, eta=0.01, tol=0.1)

Done kernel generating
iter 0 train_accuracy:0.520225
iter 2000 train_accuracy:0.636969
iter 4000 train_accuracy:0.629800
iter 6000 train_accuracy:0.636457
iter 8000 train_accuracy:0.631848
iter 10000 train_accuracy:0.637993
iter 12000 train_accuracy:0.625704
iter 14000 train_accuracy:0.635945
iter 16000 train_accuracy:0.628264
iter 18000 train_accuracy:0.641065
iter 20000 train_accuracy:0.638505
iter 22000 train_accuracy:0.634409
iter 24000 train_accuracy:0.639017
iter 26000 train_accuracy:0.625704
iter 28000 train_accuracy:0.623656
iter 30000 train_accuracy:0.638505
iter 32000 train_accuracy:0.642601
iter 34000 train_accuracy:0.636969
iter 36000 train_accuracy:0.641577
iter 38000 train_accuracy:0.640041
iter 40000 train_accuracy:0.642601
iter 42000 train_accuracy:0.631336
iter 44000 train_accuracy:0.614951
iter 46000 train_accuracy:0.629288
iter 48000 train_accuracy:0.639017
Done training, using time:(min) 2.3245592951774596
Done training
[-0.37302478 -0.64716014 -0.40195197 ...  0.5

In [14]:
experiment(gaussian_kernel, X, y, X_test, y_test, regu_para=0.01, eta=0.01, tol=0.1)

Done kernel generating
iter 0 train_accuracy:0.520225
iter 2000 train_accuracy:0.691756
iter 4000 train_accuracy:0.731695
iter 6000 train_accuracy:0.755248
iter 8000 train_accuracy:0.772145
iter 10000 train_accuracy:0.776242
iter 12000 train_accuracy:0.780850
iter 14000 train_accuracy:0.788530
iter 16000 train_accuracy:0.792627
iter 18000 train_accuracy:0.783410
iter 20000 train_accuracy:0.794163
iter 22000 train_accuracy:0.798771
iter 24000 train_accuracy:0.792627
iter 26000 train_accuracy:0.808500
iter 28000 train_accuracy:0.807476
iter 30000 train_accuracy:0.807476
iter 32000 train_accuracy:0.806964
iter 34000 train_accuracy:0.797235
iter 36000 train_accuracy:0.792115
iter 38000 train_accuracy:0.798771
iter 40000 train_accuracy:0.805428
iter 42000 train_accuracy:0.803891
iter 44000 train_accuracy:0.807476
iter 46000 train_accuracy:0.810548
iter 48000 train_accuracy:0.801331
Done training, using time:(min) 2.3260084867477415
Done training
[ -8.21414382 -14.62627928  -9.99124797 ...  

In [15]:
experiment(gaussian_kernel, X, y, X_test, y_test, regu_para=0.03, eta=0.01, tol=0.1)

Done kernel generating
iter 0 train_accuracy:0.520225
iter 2000 train_accuracy:0.681004
iter 4000 train_accuracy:0.715310
iter 6000 train_accuracy:0.724014
iter 8000 train_accuracy:0.722990
iter 10000 train_accuracy:0.727599
iter 12000 train_accuracy:0.725550
iter 14000 train_accuracy:0.721966
iter 16000 train_accuracy:0.728623
iter 18000 train_accuracy:0.729647
iter 20000 train_accuracy:0.710189
iter 22000 train_accuracy:0.729135
iter 24000 train_accuracy:0.708141
iter 26000 train_accuracy:0.729135
iter 28000 train_accuracy:0.732719
iter 30000 train_accuracy:0.726575
iter 32000 train_accuracy:0.727087
iter 34000 train_accuracy:0.727599
iter 36000 train_accuracy:0.726062
iter 38000 train_accuracy:0.731695
iter 40000 train_accuracy:0.728623
iter 42000 train_accuracy:0.726575
iter 44000 train_accuracy:0.731183
iter 46000 train_accuracy:0.730671
iter 48000 train_accuracy:0.731695
Done training, using time:(min) 2.2934930960337323
Done training
[-3.30684866 -5.28430253 -3.1607715  ...  5.8

In [16]:
experiment(gaussian_kernel, X, y, X_test, y_test, regu_para=0.003, eta=0.01, tol=0.1)

Done kernel generating
iter 0 train_accuracy:0.520225
iter 2000 train_accuracy:0.692780
iter 4000 train_accuracy:0.741423
iter 6000 train_accuracy:0.764465
iter 8000 train_accuracy:0.782898
iter 10000 train_accuracy:0.794163
iter 12000 train_accuracy:0.805428
iter 14000 train_accuracy:0.819764
iter 16000 train_accuracy:0.829493
iter 18000 train_accuracy:0.831541
iter 20000 train_accuracy:0.839734
iter 22000 train_accuracy:0.844854
iter 24000 train_accuracy:0.851510
iter 26000 train_accuracy:0.843830
iter 28000 train_accuracy:0.844854
iter 30000 train_accuracy:0.864311
iter 32000 train_accuracy:0.863287
iter 34000 train_accuracy:0.856631
iter 36000 train_accuracy:0.871480
iter 38000 train_accuracy:0.869432
iter 40000 train_accuracy:0.874552
iter 42000 train_accuracy:0.874552
iter 44000 train_accuracy:0.875576
iter 46000 train_accuracy:0.878136
iter 48000 train_accuracy:0.877624
Done training, using time:(min) 2.3400856653849282
Done training
[-17.32313689 -35.82243934 -26.53497967 ...  

In [17]:
experiment(gaussian_kernel, X, y, X_test, y_test, regu_para=0.03, eta=0.001, tol=0.01)

Done kernel generating
iter 0 train_accuracy:0.520225
iter 2000 train_accuracy:0.623144
iter 4000 train_accuracy:0.636969
iter 6000 train_accuracy:0.651306
iter 8000 train_accuracy:0.654378
iter 10000 train_accuracy:0.658986
iter 12000 train_accuracy:0.667179
iter 14000 train_accuracy:0.671275
iter 16000 train_accuracy:0.676395
iter 18000 train_accuracy:0.679980
iter 20000 train_accuracy:0.685612
iter 22000 train_accuracy:0.690732
iter 24000 train_accuracy:0.685100
iter 26000 train_accuracy:0.690732
iter 28000 train_accuracy:0.698413
iter 30000 train_accuracy:0.696877
iter 32000 train_accuracy:0.695853
iter 34000 train_accuracy:0.701997
iter 36000 train_accuracy:0.698925
iter 38000 train_accuracy:0.705069
iter 40000 train_accuracy:0.705581
iter 42000 train_accuracy:0.711214
iter 44000 train_accuracy:0.710701
iter 46000 train_accuracy:0.711726
iter 48000 train_accuracy:0.712750
Done training, using time:(min) 2.387454128265381
Done training
[-2.74388166 -4.2579082  -2.46283814 ...  4.63

## sigma = 10

In [21]:
experiment(gaussian_kernel, X, y, X_test, y_test, regu_para=0.1, eta=0.01, tol=0.01)

Done kernel generating
iter 0 train_accuracy:0.548387
iter 2000 train_accuracy:0.896057
iter 4000 train_accuracy:0.905786
iter 6000 train_accuracy:0.907322
iter 8000 train_accuracy:0.907834
iter 10000 train_accuracy:0.906810
iter 12000 train_accuracy:0.908346
iter 14000 train_accuracy:0.908858
iter 16000 train_accuracy:0.907834
iter 18000 train_accuracy:0.905274
iter 20000 train_accuracy:0.908346
iter 22000 train_accuracy:0.906810
iter 24000 train_accuracy:0.905786
iter 26000 train_accuracy:0.905274
iter 28000 train_accuracy:0.907322
iter 30000 train_accuracy:0.910906
iter 32000 train_accuracy:0.911418
iter 34000 train_accuracy:0.908346
iter 36000 train_accuracy:0.905786
iter 38000 train_accuracy:0.907834
iter 40000 train_accuracy:0.906298
iter 42000 train_accuracy:0.905786
iter 44000 train_accuracy:0.902714
iter 46000 train_accuracy:0.908858
iter 48000 train_accuracy:0.909882
Done training, using time:(min) 2.656913165251414
Done training
[-0.85800527 -1.4069801  -1.08640482 ...  1.44

In [ ]:
experiment(gaussian_kernel, X, y, X_test, y_test, regu_para=0.03, eta=0.01, tol=0.01)

Done kernel generating
iter 0 train_accuracy:0.546339
iter 2000 train_accuracy:0.944188
iter 4000 train_accuracy:0.971326
iter 6000 train_accuracy:0.978495
iter 8000 train_accuracy:0.981567
iter 10000 train_accuracy:0.983103
iter 12000 train_accuracy:0.984127


In [ ]:
experiment(gaussian_kernel, X, y, X_test, y_test, regu_para=0.01, eta=0.01, tol=0.01)

In [ ]:
experiment(gaussian_kernel, X, y, X_test, y_test, regu_para=0.3, eta=0.01, tol=0.01)

# inhomogeneous kernel exps

In [ ]:
experiment(inhomogeneous_poly_kernel, X, y, X_test, y_test, regu_para=0, eta=0.001, tol=0.01)

Done kernel generating
iter 0 train_accuracy:0.525858


/home/y247xie/.local/lib/python3.5/site-packages/ipykernel_launcher.py:3: RuntimeWarning: overflow encountered in exp
  This is separate from the ipykernel package so we can avoid doing imports until


iter 2000 train_accuracy:0.962110
iter 4000 train_accuracy:0.988223
Done training, using time:(min) 0.28302624622980754
Done training
[-0.044  -0.2965 -0.127  ...  0.266   0.1635  0.26  ]
training result
0.9912954429083461
test result
0.6015


In [ ]:
experiment(inhomogeneous_poly_kernel, X, y, X_test, y_test, regu_para=1, eta=0.001, tol=0.1)

Done kernel generating
iter 0 train_accuracy:0.518689


/home/y247xie/.local/lib/python3.5/site-packages/ipykernel_launcher.py:3: RuntimeWarning: overflow encountered in exp
  This is separate from the ipykernel package so we can avoid doing imports until


iter 2000 train_accuracy:0.544291
iter 4000 train_accuracy:0.570405
iter 6000 train_accuracy:0.568356
iter 8000 train_accuracy:0.594982
iter 10000 train_accuracy:0.586790
iter 12000 train_accuracy:0.699949
iter 14000 train_accuracy:0.648746
iter 16000 train_accuracy:0.830005
iter 18000 train_accuracy:0.526370
iter 20000 train_accuracy:0.644137
iter 22000 train_accuracy:0.573477
iter 24000 train_accuracy:0.578085
iter 26000 train_accuracy:0.811572
iter 28000 train_accuracy:0.658986
iter 30000 train_accuracy:0.653354
iter 32000 train_accuracy:0.796723
iter 34000 train_accuracy:0.813620
iter 36000 train_accuracy:0.734767
iter 38000 train_accuracy:0.712750
iter 40000 train_accuracy:0.547363
iter 42000 train_accuracy:0.539171
iter 44000 train_accuracy:0.631848
iter 46000 train_accuracy:0.687660
iter 48000 train_accuracy:0.830517
Done training, using time:(min) 2.8351170420646667
Done training
[-0.02731652 -0.18705636 -0.09454211 ...  0.16458566  0.12203252
  0.17388478]
training result
0.63

In [ ]:
experiment(inhomogeneous_poly_kernel, X, y, X_test, y_test, regu_para=10, eta=0.01, tol=0.1)

Done kernel generating
iter 0 train_accuracy:0.561700


/home/y247xie/.local/lib/python3.5/site-packages/ipykernel_launcher.py:3: RuntimeWarning: overflow encountered in exp
  This is separate from the ipykernel package so we can avoid doing imports until


iter 2000 train_accuracy:0.521249
iter 4000 train_accuracy:0.524322
iter 6000 train_accuracy:0.520737
iter 8000 train_accuracy:0.529954
iter 10000 train_accuracy:0.522273
iter 12000 train_accuracy:0.521761
iter 14000 train_accuracy:0.503328
iter 16000 train_accuracy:0.479775
iter 18000 train_accuracy:0.519713
iter 20000 train_accuracy:0.521761
iter 22000 train_accuracy:0.521761
iter 24000 train_accuracy:0.532002
iter 26000 train_accuracy:0.522273
iter 28000 train_accuracy:0.494112
iter 30000 train_accuracy:0.521249
iter 32000 train_accuracy:0.521761
iter 34000 train_accuracy:0.483359
iter 36000 train_accuracy:0.519201
iter 38000 train_accuracy:0.520225
iter 40000 train_accuracy:0.522273
iter 42000 train_accuracy:0.516129
iter 44000 train_accuracy:0.494112
iter 46000 train_accuracy:0.492063
iter 48000 train_accuracy:0.519713
Done training, using time:(min) 2.832144820690155
Done training
[-0.00625517 -0.0331439  -0.01440623 ...  0.01616075  0.02451547
  0.018909  ]
training result
0.520

In [ ]:
experiment(inhomogeneous_poly_kernel, X, y, X_test, y_test, regu_para=10, eta=0.001, tol=0.1)

In [ ]:
experiment(inhomogeneous_poly_kernel, X, y, X_test, y_test, regu_para=1, eta=0.03, tol=0.1)